In [ ]:
import pandas as pd
import requests
import json
import numpy as np

In [ ]:
#stock_data=pd.read_csv("./stock_data.csv")
stock_data=pd.read_csv("./stock_data_all.csv")

In [ ]:
stock_data.shape

In [ ]:
stock_df={}
for col in range(stock_data.shape[1]):
    if col>0:
        stock_df[stock_data.columns[col]]=[]

In [ ]:
means_list=[np.nan]
for col in range(stock_data.shape[1]):
    if col>0:
        dt=stock_data[stock_data.columns[col]][stock_data[stock_data.columns[col]].isnull()==False].reset_index()
        colmean=np.mean([float(eval(dt.iloc[i,1])['4. close']) for i in range(dt.shape[0]) ])
        means_list.append(colmean)

In [ ]:
for col in range(stock_data.shape[1]):
    for row in range(stock_data.shape[0]):
        if col>0:
            try:
                stock_df[stock_data.columns[col]].append(eval(stock_data.iloc[row,col])['4. close'])
            except TypeError:
                print(f"{row},{col}")
                #stock_df[stock_data.columns[col]].append(stock_df[stock_data.columns[col]][-1])
                stock_df[stock_data.columns[col]].append(means_list[col])

In [ ]:
df=pd.DataFrame(stock_df)

df['Date']=stock_data.iloc[:,0]

In [ ]:
df.shape

In [ ]:
df2=pd.melt(df, id_vars=['Date'], value_vars=stock_data.columns[1:])

df2=df2.rename(columns={'Date':'Close_Date',
             'variable':'Symbol',
             'value':'Close_Price'})

In [ ]:
df2['Close_Date']=pd.to_datetime(df2['Close_Date'])

df2=df2.sort_values(['Symbol','Close_Date'])

df2['Close_Price_Next_Day'] = df2.groupby(['Symbol'])['Close_Price'].shift(-1)

df2=df2[df2['Close_Price_Next_Day'].notnull()]

df2['Daily_Return']=pd.to_numeric(df2['Close_Price_Next_Day'])/pd.to_numeric(df2['Close_Price'])

In [ ]:
stock_quantiles={}
stock_quantiles['lower_limit']=df2.groupby(['Symbol'])['Daily_Return'].quantile(.1)
stock_quantiles['upper_limit']=df2.groupby(['Symbol'])['Daily_Return'].quantile(.9)
quantile_df=pd.DataFrame(stock_quantiles).reset_index()


In [ ]:
clean_df=df2.merge(quantile_df, on=['Symbol'], how='left')
clean_df['Action_for_News_Date']='Hold'
clean_df.loc[(clean_df['Daily_Return'] >clean_df['lower_limit']) & (clean_df['Daily_Return'] <clean_df['upper_limit']), 'Action_for_News_Date'] = 'Hold'
clean_df.loc[(clean_df['Daily_Return'] <=clean_df['lower_limit']), 'Action_for_News_Date'] = 'Sell'
clean_df.loc[(clean_df['Daily_Return'] >=clean_df['upper_limit']), 'Action_for_News_Date'] = 'Buy'

In [ ]:
clean_df=clean_df[['Close_Date','Symbol','Action_for_News_Date']]

clean_df=clean_df.pivot(index='Close_Date', columns='Symbol', values='Action_for_News_Date')

clean_df=clean_df.reset_index()

In [ ]:
# clean_df.iloc[0,:].value_counts()['Hold']
# clean_df['Hold']=0
# clean_df['Buy']=0
# clean_df['Sell']=0
# for row in range(clean_df.shape[0]):
#     try:
#         clean_df['Hold'][row]=clean_df.iloc[row,:].value_counts()['Hold']
#         clean_df['Buy'][row]=clean_df.iloc[row,:].value_counts()['Buy']
#         clean_df['Sell'][row]=clean_df.iloc[row,:].value_counts()['Sell']
#     except KeyError:
#         print(row)
      

In [ ]:
clean_df.iloc[0,:].value_counts()['Hold']

clean_df['Hold']=0
clean_df['Buy']=0
clean_df['Sell']=0
for row in range(clean_df.shape[0]):
    hold=0
    buy=0
    sell=0
    for col in range(clean_df.shape[1]):
        if (clean_df.iloc[row,col]=="Hold"):
            hold+=1
        elif (clean_df.iloc[row,col]=="Buy"):
            buy+=1
        else:
            sell+=1
                #simul_clean_df['Hold'][row]=simul_clean_df.iloc[row,:].value_counts()['Hold']
        #simul_clean_df['Buy'][row]=simul_clean_df.iloc[row,:].value_counts()['Buy']
        #simul_clean_df['Sell'][row]=simul_clean_df.iloc[row,:].value_counts()['Sell']
        
    clean_df.loc[row,'Hold']=hold
    clean_df.loc[row,'Buy']=buy
    clean_df.loc[row,'Sell']=sell  

In [ ]:
clean_df.head()

In [ ]:
clean_df['check']=clean_df['Hold']+clean_df['Buy']+clean_df['Sell']

clean_df=clean_df[['Close_Date','Hold','Buy','Sell']]

In [ ]:
clean_df['Buy']=(clean_df['Buy']//2)*2
clean_df['Sell']==(clean_df['Sell']//2)*2

In [ ]:
db_df=pd.read_csv("Data/Conditional_Buy_Quantile.csv")
ds_df=pd.read_csv("Data/Conditional_Sell_Quantile.csv")
db_df.loc[db_df['Sell']==116,:]

clean_df=clean_df.merge(db_df, on=['Sell'], how='left')
clean_df=clean_df.merge(ds_df, on=['Buy'], how='left')

In [ ]:
clean_df=clean_df[['Close_Date','Hold','Buy','buy_q_0.90','Sell','sell_q_0.90']]

In [ ]:
clean_df['Action_for_News_Date']="Hold"
for row in range(clean_df.shape[0]):
    if (clean_df.loc[row,'Buy']>=clean_df.loc[row,'buy_q_0.90']):
        if (clean_df.loc[row,'Sell']>=clean_df.loc[row,'sell_q_0.90']):
            if (clean_df.loc[row,'Buy']>=clean_df.loc[row,'Sell']):
                clean_df.loc[row,'Action_for_News_Date']="Buy"
            else:
                clean_df.loc[row,'Action_for_News_Date']="Sell"
        else:
            clean_df.loc[row,'Action_for_News_Date']="Buy"
    elif (clean_df.loc[row,'Sell']>=clean_df.loc[row,'sell_q_0.90']):
        clean_df.loc[row,'Action_for_News_Date']='Sell'
    else:
        clean_df.loc[row,'Action_for_News_Date']="Hold"
        
            

In [ ]:
clean_df=clean_df[['Close_Date','Action_for_News_Date']]

In [ ]:
news_stories={}
news_stories['Date']=[]
news_stories['content']=[]
news_stories['title']=[]
news_stories['category']=[]
news_stories['id']=[]
news_stories['sourceurl']=[]
news_stories['imageurl']=[]

In [ ]:
with open("../Data/articles.json") as jsonFile:
   sourceData = json.load(jsonFile)
for article in sourceData['articles']:
    #print(article)
    date=str(article['publishdate'])
    news_stories['Date'].append(date[0:4]+"-"+date[4:6]+"-"+date[6:8])
    news_stories['content'].append(article['content'])
    news_stories['title'].append(article['title'])
    news_stories['category'].append(article['category'])
    news_stories['id'].append(article['id'])
    news_stories['sourceurl'].append(article['sourceurl'])
    news_stories['imageurl'].append(article['imageurl'])

In [ ]:
news_stories_df=pd.DataFrame(news_stories)
news_stories_df['Date']=pd.to_datetime(news_stories_df['Date'])

In [ ]:
news_stories_labelled=news_stories_df.merge(clean_df,left_on='Date',right_on='Close_Date',how="inner")

In [ ]:
import time

labelled_stories={}
labelled_stories['createdate']=time.strftime("%Y-%m-%d")
labelled_stories['articles']=[]
for row in range(news_stories_labelled.shape[0]):
    article={}
    article['publishdate']=str(news_stories_labelled.loc[row,'Date'].date().year)+\
    "-"+str(news_stories_labelled.loc[row,'Date'].date().month).rjust(2,'0')+"-"+\
    str(news_stories_labelled.loc[row,'Date'].date().day).rjust(2,'0')
    article['content']=news_stories_labelled.loc[row,'content']
    article['label']=news_stories_labelled.loc[row,'Action_for_News_Date']
    article['title']=news_stories_labelled.loc[row,'title']
    article['category']=news_stories_labelled.loc[row,'category']
    article['id']=news_stories_labelled.loc[row,'id']
    article['sourceurl']=news_stories_labelled.loc[row,'sourceurl']
    article['imageurl']=news_stories_labelled.loc[row,'imageurl']
    labelled_stories['articles'].append(article)

In [ ]:
print("labelled_stories keys")
print(labelled_stories.keys())
print("article keys")
print(labelled_stories['articles'][0].keys())
print("number of articles")
print(len(labelled_stories['articles']))

In [ ]:
with open('3.News_With_Labels.json', 'w') as outfile:  
    json.dump(labelled_stories, outfile)

In [ ]:
with open('../Data/3.News_With_Labels.json', 'w') as outfile:  
    json.dump(labelled_stories, outfile)